In [1]:
from torchvision.models import *
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'CAPTCHA-Images'

In [2]:
def load_data():
    words = []
    data = []
    for word in os.listdir('./data/'):
        word = word.split('.')
        word = word[0]
        for letter in word:
            words.append(letter)
    words = sorted(set(words))
    for file in os.listdir('./data/'):
        img = cv2.imread(f'./data/{file}')
        img = cv2.resize(img,(56,56)) / 255.0
        word = file.split('.')
        word = word[0]
        letters = []
        for letter in word:
            letters.append(letter)
        bag = np.zeros(len(words))
        for idx,w in enumerate(words):
            if w in letters:
                bag[idx] = 1.0
        data.append([img,bag])
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)
    X_train = torch.from_numpy(np.array(X_train)).to(device).view(-1,3,56,56).float()
    y_train = torch.from_numpy(np.array(y_train)).to(device).float()
    X_test = torch.from_numpy(np.array(X_test)).to(device).view(-1,3,56,56).float()
    y_test = torch.from_numpy(np.array(y_test)).to(device).float()
    return X,y,X_train,X_test,y_train,y_test,data,words

In [3]:
X,y,X_train,X_test,y_train,y_test,data,words = load_data()

In [4]:
torch.save(X_train,'X_train.pt')
torch.save(y_train,'y_train.pt')
torch.save(X_test,'X_test.pt')
torch.save(y_test,'y_test.pt')
torch.save(X_train,'X_train.pth')
torch.save(y_train,'y_train.pth')
torch.save(X_test,'X_test.pth')
torch.save(y_test,'y_test.pth')
torch.save(words,'words.pth')
torch.save(words,'words.pth')

In [5]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [6]:
def matrix_to_words(words,martrix):
    matrix_word = ''
    for idx,m in enumerate(martrix):
        m = round(float(m))
        if m == 1.0 or m == 1:
            matrix_word  = matrix_word + words[idx]
    return matrix_word

In [7]:
def get_accuracy_1(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        for p,yb in zip(pred,y_batch):
            p = int(p)
            yb = int(yb)
            if p == yb:
                correct += 1
                total += 1
    acc = round(correct/total,3)*100
    return acc

In [8]:
def get_accuracy_2(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = matrix_to_words(words,pred)
        y_batch = matrix_to_words(words,y_batch)
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [9]:
model = resnet34().to(device)

In [10]:
model.fc = Linear(512,len(words))

In [11]:
criterion = MSELoss()

In [12]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [13]:
batch_size = 32

In [14]:
epochs = 1000

In [15]:
matrix_to_words(words,y[0])

'7fgm'

In [16]:
y[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 0.])

In [17]:
import time

In [18]:
import random

In [19]:
epochs_iter = tqdm(range(epochs))
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in epochs_iter:
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        model.to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    for file in os.listdir('./data/'):
        img = cv2.imread(f'./data/{file}')
        img = cv2.resize(img,(56,56))
        preds = model(torch.from_numpy(np.array(img)).to(device).view(-1,3,56,56).float())
        epochs_iter.set_description(f'{file}/{matrix_to_words(words,preds[0])}')
        break
    model.train()
    torch.cuda.empty_cache()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy_1(model,X_train,y_train)+get_accuracy_2(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val ACC':(get_accuracy_1(model,X_test,y_test)+get_accuracy_2(model,X_test,y_test))/2})
    torch.cuda.empty_cache()
    time.sleep(5)
    for file in os.listdir('./data/'):
        img = cv2.imread(f'./data/{file}')
        img = cv2.resize(img,(56,56))/255.0
        preds = model(torch.from_numpy(np.array(img)).to(device).view(-1,3,56,56).float())
        epochs_iter.set_description(f'{file}/{matrix_to_words(words,preds[0])}')
        break
wandb.finish()

  0%|                                                  | 0/1000 [00:00<?, ?it/s]wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/indika/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
fg7mg.png/c:  18%|████▌                    | 180/1000 [19:42<1:29:45,  6.57s/it]


KeyboardInterrupt: 

Error in callback <function _WandbInit._pause_backend at 0x7faf50edc710> (for post_run_cell):


Process wandb_internal:
Traceback (most recent call last):
  File "/home/indika/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/indika/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/internal/internal.py", line 152, in wandb_internal
    thread.join()
  File "/home/indika/anaconda3/lib/python3.7/threading.py", line 1044, in join
    self._wait_for_tstate_lock()
  File "/home/indika/anaconda3/lib/python3.7/threading.py", line 1060, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
torch.save(model,'model-resnet18.pt')
torch.save(model,'model-resnet18.pth')
torch.save(model.state_dict(),'model-sd-resnet18.pt')
torch.save(model.state_dict(),'model-sd-resnet18.pth')

In [ ]:
def matrix_to_words(words,martrix):
    matrix_word = ''
    for idx,m in enumerate(martrix):
#         print(m)
        m = round(float(m))
        if m == 1.0 or m == 1:
            matrix_word  = matrix_word + words[idx]
    return matrix_word